In [1]:
setwd("/frazer01/projects/CARDIPS/analysis/cardiac_eqtls")

source("script/packages.R"  )
source("script/input_data.R")
source("script/functions.R" )


In [17]:
gene_info               = fread("input/phenotypes/gene_info.txt"   , sep = "\t", header = TRUE, data.table = FALSE)
isof_info               = fread("input/phenotypes/isoform_info.txt", sep = "\t", header = TRUE, data.table = FALSE)
gene_info$transcript_id = gene_info$gene_id
promoters               = fread("/frazer01/reference/private/Gencode.v34lift37/promoters.bed", sep = "\t", header = FALSE, data.table = FALSE)


In [10]:
diffexp         = fread("pipeline/4.1.differential_expression/diffexp.txt", sep = "\t", header = TRUE, data.table = FALSE)


In [11]:
tmpwd = "pipeline/5.1.expression_enrichments/stage_enrichment"
dir.create(tmpwd, showWarnings = FALSE)

In [12]:
tissues = unique(diffexp[,c("tissue1", "tissue2")])

tissues

,tissue1,tissue2
,<chr>,<chr>
1,ipsc_cvpc,heart
130468,ipsc_cvpc,arteria
260935,heart,arteria


In [25]:
run_homer_tissue = function(tissue, name, genes, isof_info, promoters)
{
    isoforms           = promoters[ promoters[,4] %in% isof_info[ isof_info$gene_id %in% genes | isof_info$transcript_id %in% genes, "transcript_id"],]
    colnames(isoforms) = c("chrom", "from", "to", "transcript_id")
    isoforms           = merge(isoforms, isof_info[,c("transcript_id", "strand")])
    isoforms$score     = 0
    isoforms           = isoforms[order(isoforms[,"chrom"], isoforms[,"from"], isoforms[,"to"]),]
    analysis_name      = paste(name, tissue, sep = ".")
    bed_file           = paste(tmpwd, paste(analysis_name,             "bed", sep = "."), sep = "/")
    bed_file_unmerged  = paste(tmpwd, paste(analysis_name, "unmerged", "bed", sep = "."), sep = "/")
    motif_file         = "/frazer01/reference/public/hocomoco_v11/HOCOMOCOv11_core_HUMAN_mono_homer_format_0.0001.motif"

    fwrite(isoforms[,c("chrom", "from", "to", "transcript_id", "score", "strand")], bed_file_unmerged, sep = "\t", col.names = FALSE, row.names = FALSE)
    
    system(paste("bedtools merge -c 4 -o distinct", "-i", bed_file_unmerged, ">", bed_file))
    
    command = paste("findMotifsGenome.pl", bed_file, "hg19", paste(tmpwd, analysis_name, sep = "/"), 
                    "-mknown" , motif_file, 
                    "-mcheck" , motif_file, 
                    "-nomotif", 
                    "-size"   , 200,
                    ""
                   )
    
    return(command)
}

enrichment_epigenome = function(diffexp, tissue1, tissue2, type, isof_info, promoters)
{
    message(paste(tissue1, tissue2, type))
    
    x  = diffexp[diffexp$type == type & diffexp$tissue1 == tissue1 & diffexp$tissue2 == tissue2 & diffexp$diffexp == TRUE, ]
    
    dif1 = x[x$beta > 0,]
    dif2 = x[x$beta < 0,]
    
    out     = c(run_homer_tissue(tissue1, paste(type, tissue1, tissue2, sep = "."), unique(dif1$transcript_id), isof_info, promoters),
                run_homer_tissue(tissue2, paste(type, tissue1, tissue2, sep = "."), unique(dif2$transcript_id), isof_info, promoters),
                "")
    return(out)
}

to_run = unlist(lapply(1:nrow(tissues), function(ii)
{
    tissue1 = tissues[ii, "tissue1"]
    tissue2 = tissues[ii, "tissue2"]
    
    out2 = unlist(lapply(c("gene_tpm", "isoform_use"), function(type)
    {
        enrichment_epigenome(diffexp, tissue1, tissue2, type, isof_info, promoters)
    }))
    return(out2)
}))


ipsc_cvpc heart gene_tpm

ipsc_cvpc heart isoform_use

ipsc_cvpc arteria gene_tpm

ipsc_cvpc arteria isoform_use

heart arteria gene_tpm

heart arteria isoform_use



In [26]:
shfile = paste(tmpwd, "run_homer.sh", sep = ".")
writeLines(to_run, con = shfile, sep = "\n")

message(paste("bash", shfile))
message(paste("nohup", "bash", shfile, "&"))

bash pipeline/5.1.expression_enrichments/stage_enrichment.run_homer.sh

nohup bash pipeline/5.1.expression_enrichments/stage_enrichment.run_homer.sh &



In [27]:
motifs

ERROR: Error in eval(expr, envir, enclos): object 'motifs' not found
